In [1]:
from docling.chunking import HierarchicalChunker
from docling.document_converter import DocumentConverter

c:\Users\dinhd\miniconda3\envs\notebook\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
converter = DocumentConverter()
result = converter.convert("./test.pdf")
doc = result.document

2025-12-30 15:08:11,579 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-30 15:08:11,584 - INFO - Going to convert document batch...
2025-12-30 15:08:11,584 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e15bc6f248154cc62f8db15ef18a8ab7
2025-12-30 15:08:11,585 - INFO - Accelerator device: 'cpu'
[INFO] 2025-12-30 15:08:11,605 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-12-30 15:08:11,611 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\dinhd\miniconda3\envs\notebook\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-12-30 15:08:11,612 [RapidOCR] main.py:53: Using C:\Users\dinhd\miniconda3\envs\notebook\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-12-30 15:08:11,696 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-12-30 15:08:11,697 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\dinhd\miniconda3\envs\notebook\Lib\site-packag

In [21]:
def show_texts(doc, limit=20):
    for i, t in enumerate(doc.texts[:limit]):
        print(f"[{i}] {t.text[:80]}")

show_texts(doc)

[0] CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM Độc lập - Tự do - Hạnh phúc
[1] ***
[2] LÝ LỊCH KHOA HỌC
[3] I. LÝ LỊCH SƠ LƯỢC
[4] Họ và tên: Nguyễn Đình Đạt Giới tính: Nam Ngày, tháng, năm sinh: 20/11/2005 Nơi 
[5] II. QUÁ TRÌNH ĐÀO TẠO
[6] Đại học:
[7] Hệ đào tạo: Chính quy Nơi đào tạo: Viện Khoa học Kỹ thuật Bưu điện - Học viện Cô
[8] 2.
[9] III. QUÁ TRÌNH CÔNG TÁC CHUYÊN MÔN
[10] IV. QUÁ TRÌNH NGHIÊN CỨU KHOA HỌC
[11] Hà Nội, ngày   tháng   năm 2025
[12] Các đề tài nghiên cứu khoa học đã và đang tham gia
[13] 2. Các công trình khoa học đã và đang công bố
[14] 3. Giải thưởng khoa học và công nghệ
[15] XÁC NHẬN CƠ SỞ ĐÀO TẠO Hà Nội., ngày     tháng    năm 2025 Người khai kí tên
[16] Nguyễn Đình Đạt
[17] Dac diem nhan dang/Personalidentication:
[18] NotruoiC2cmduoi truocduoi
[19] matphai


In [20]:
doc

DoclingDocument(schema_name='DoclingDocument', version='1.8.0', name='test', origin=DocumentOrigin(mimetype='application/pdf', binary_hash=9229799015699733589, filename='test.pdf', uri=None), furniture=GroupItem(self_ref='#/furniture', parent=None, children=[], content_layer=<ContentLayer.FURNITURE: 'furniture'>, meta=None, name='_root_', label=<GroupLabel.UNSPECIFIED: 'unspecified'>), body=GroupItem(self_ref='#/body', parent=None, children=[RefItem(cref='#/texts/0'), RefItem(cref='#/texts/1'), RefItem(cref='#/texts/2'), RefItem(cref='#/texts/3'), RefItem(cref='#/texts/4'), RefItem(cref='#/texts/5'), RefItem(cref='#/groups/0'), RefItem(cref='#/texts/7'), RefItem(cref='#/groups/1'), RefItem(cref='#/texts/9'), RefItem(cref='#/tables/0'), RefItem(cref='#/texts/10'), RefItem(cref='#/texts/11'), RefItem(cref='#/groups/2'), RefItem(cref='#/tables/1'), RefItem(cref='#/texts/13'), RefItem(cref='#/tables/2'), RefItem(cref='#/tables/3'), RefItem(cref='#/texts/14'), RefItem(cref='#/tables/4'), Re

In [6]:
chunker = HierarchicalChunker()
chunks = chunker.chunk(doc)

In [19]:
for chunk in chunks:
    print(chunk.content)

In [13]:
from anytree import Node, RenderTree, Resolver
from anytree.resolver import ChildResolverError

def build_tree_with_anytree(chunks):
    root = Node("Document_Root", content="")
    resolver = Resolver('name')

    for chunk in chunks:
        headings = chunk.meta.headings
        current_node = root
        
        # Duyệt qua từng cấp độ heading để tạo/tìm node
        path = ""
        for h_name in headings:
            # Xử lý tên node để tránh ký tự đặc biệt gây lỗi đường dẫn anytree
            safe_name = h_name.replace("/", "-").strip()
            
            try:
                # Thử tìm xem node đã tồn tại chưa
                current_node = resolver.get(current_node, safe_name)
            except ChildResolverError:
                # Nếu chưa có thì tạo mới
                current_node = Node(safe_name, parent=current_node, content="")
        
        # Gán nội dung vào node lá (hoặc nối thêm nếu nốt đó đã có nội dung)
        if current_node.content:
            current_node.content += "\n" + chunk.text
        else:
            current_node.content = chunk.text
            
    return root

doc_tree = build_tree_with_anytree(chunks)

In [16]:
for pre, fill, node in RenderTree(doc_tree):
    content_snippet = (node.content[:50] + "..") if node.content else " (Heading Node)"
    print(f"{pre}{node.name} | Len: {len(node.content)} chars")

Document_Root | Len: 0 chars
